# CLEAN CSVs

In [1]:
# Dependencies
import datetime as dt
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date, create_engine, func, create_engine, MetaData

from flask import Flask, jsonify

In [2]:
# Path of the CSV file
csvmeasurments = "hawaii_measurements.csv"
csvstations = "hawaii_stations.csv"

In [3]:
# Read CSV file into a pandas DataFrame
df1 = pd.read_csv(csvmeasurments)
df1.dropna(axis=0)
df2 = pd.read_csv(csvstations)

In [4]:
# Export clean data to CSV
df1.to_csv('clean_measurements.csv')
df2.to_csv('clean_stations.csv')

# Database Creation

In [5]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [7]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.

Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Integer)
    tobs = Column(Integer)
 
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Stations(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    

In [8]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [9]:
# Load the cleaned csv file into a pandas dataframe
clean_stations = "clean_stations.csv"
new_df = pd.read_csv(clean_stations)

clean_measurements = "clean_measurements.csv"
new_df2 = pd.read_csv(clean_measurements)

In [10]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
stations_data = new_df.to_dict(orient='records')
measurements_data = new_df2.to_dict(orient='records')

In [11]:
print(stations_data[:5])

[{'Unnamed: 0': 0, 'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'Unnamed: 0': 1, 'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'Unnamed: 0': 2, 'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'Unnamed: 0': 3, 'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'Unnamed: 0': 4, 'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [12]:
print(measurements_data[:5])

[{'Unnamed: 0': 0, 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'Unnamed: 0': 1, 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'Unnamed: 0': 2, 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'Unnamed: 0': 3, 'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'Unnamed: 0': 4, 'station': 'USC00519397', 'date': '2010-01-06', 'prcp': nan, 'tobs': 73}]


In [13]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [14]:
# Save the reference to the `stations` table as a variable called `table`
table = sqlalchemy.Table('measurements', metadata, autoload=True)
table2 = sqlalchemy.Table('stations', metadata, autoload=True)

In [15]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())
conn.execute(table2.delete())

In [16]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), measurements_data)

In [17]:
conn.execute(table2.insert(), stations_data)

In [18]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-06', None, 73)]

In [19]:
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]